In [1]:
import keras
#import keras_resnet.models
from keras.models import Model
from keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, concatenate, \
    Activation, ZeroPadding2D
from keras.layers import add, Flatten
from keras.utils import plot_model
from keras.metrics import top_k_categorical_accuracy
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.callbacks import ReduceLROnPlateau, CSVLogger,ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras import backend as K
from keras import optimizers
from keras import applications
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
#from keras.applications.vgg16 import VGG16
#from keras.applications.vgg16 import preprocess_input
#from keras.applications import vgg16
from sklearn.utils import shuffle


Using TensorFlow backend.


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
import numpy as np
import glob
import cv2
import os
data_src = '//data/examples/flower_classification/'

In [4]:
'''設定參數'''
classn = 5
batch_size = classn * 2
epochs = 80
lr = 0.0001
train_ratio= 0.8
sz=224
r_state = 40
channel = 3

In [5]:
labelmap=pd.read_csv(data_src+'mapping.csv',names=['dirs','class'])

In [6]:
sortlabel=labelmap.sort('class')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  """Entry point for launching an IPython kernel.


In [7]:
targetlist=sortlabel.dirs.as_matrix().tolist()
targetlist[:5]

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

targetlist

In [9]:
x=[]
y=[]
for i,c in enumerate(targetlist):
    ff=glob.glob((data_src+'train/'+c+'/*jpg'))
    for f in ff:
        img=cv2.imread(f) #img=cv2.imread(f, 0 )
        img=cv2.resize(img,(sz,sz))
        img=cv2.cvtColor(img, cv2.COLOR_RGB2BGR) #COLOR_GRAY2BGR
        img = img[:,:,::-1] # cv2 預設讀進來是 BGR, 我們要轉回 RGB
        # zero-mean
        # pre-trained model 使用 ImageNet 做訓練
        # ImageNet 的所有影像 RGB 平均值 [123.68, 116.78, 103.94]
        #img = img.astype('float32') - np.array([123.68, 116.78, 103.94])
        img = img.astype('float32')
        x.append(img)
        nn=np.zeros(15,dtype=float)
        nn[i]=1
        y.append(nn)
xx=np.array(x)
yy=np.array(y)

xx = preprocess_input(xx)

IndexError: too many indices for array

In [ ]:
xx.shape

In [ ]:
yy.shape

In [ ]:
#----------------------------資料training set, testing set 分割---------------------------------------

x_train, x_valid, y_train, y_valid = train_test_split(xx, 
                                                      yy, 
                                                      test_size = 1.0 - train_ratio,
                                                      stratify = y,
                                                      random_state = r_state)
#------------------------------------檢查資料---------------------------------------------------------
print("training set data dimension")
print(x_train.shape)
print(y_train.shape)
print("-----------")
print("training set: %i" % len(x_train))
print("validation set: %i" % len(x_valid))

x_train = x_train.astype('float32')
x_valid = x_valid.astype('float32')


# subtract mean and normalize
mean_image = np.mean(x_train, axis=0)
x_train -= mean_image
x_valid -= mean_image
x_train /= 255.
x_valid /= 255.

In [ ]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.01), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.01, patience=10)
mcp_save = ModelCheckpoint('best.hdf5', save_best_only=True, monitor='val_loss', mode='min')
csv_logger = CSVLogger('resnet18.csv')

In [ ]:
callbacks=[lr_reducer, early_stopper, csv_logger,mcp_save]

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=True,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        shear_range = 0.1,
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
# datagen.fit(x_train)

In [ ]:
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (sz, sz, channel))

# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(15, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer=optimizers.RMSprop(lr=lr), metrics=["accuracy"])


history = model_final.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                                    steps_per_epoch=x_train.shape[0] / batch_size,
                                    validation_data=(x_valid, y_valid),  
                                    epochs=epochs, 
                                    callbacks=[lr_reducer, early_stopper, csv_logger,mcp_save])

In [ ]:
testdata=pd.read_csv(data_src+'submission.csv')
testdata.shape

testdata

In [ ]:
testdataindexlist=testdata.id.as_matrix().tolist()
testdataindexlist[:5]

In [ ]:
x_t=[]
for f in testdataindexlist:
    img=cv2.imread(data_src+'test/'+f+'.jpg') #,1
    img=cv2.resize(img,(sz,sz))
    img=cv2.cvtColor(img, cv2.COLOR_RGB2BGR) #COLOR_GRAY2BGR
    img = img[:,:,::-1]
    #img = img.astype('float32') - np.array([123.68, 116.78, 103.94])
    img = img.astype('float32')
    x_t.append(img)

        
x_test=np.array(x_t)
x_test = preprocess_input(x_test)

In [ ]:
#%%script false
model_final.load_weights(filepath = 'best.hdf5')

score = model_final.evaluate(x_train, y_train, verbose=2)
print("Best model")
print('\t-CV loss:', score[0])
print('\t-CV accuracy:', score[1])

pt = model_final.predict(x_train)
mse = (np.mean((pt-y_train)**2))
print('\t-CV MSE: ', mse)


In [ ]:
predicts = model_final.predict(x_test, verbose=1)

In [ ]:
result = []
codeMap = {}
maxPossible = np.argmax(predicts,axis=1)
for i in range(0, len(predicts)):
    filename = testdata.id[i]
    code = maxPossible[i]
    codeMap[code] = code
#     print(i)
    result.append([filename,code])

In [ ]:
result[:5]

In [ ]:
answer=pd.DataFrame(result,columns=['id','class'])

In [ ]:
answer.to_csv('vgg19-submission.csv', index = False)